In [1]:
!pip3 install openai
!pip3 install --upgrade openai

Defaulting to user installation because normal site-packages is not writeable
--- Logging error ---
Traceback (most recent call last):
  File "/Users/joshua.grefte/Library/Python/3.10/lib/python/site-packages/pip/_internal/utils/logging.py", line 177, in emit
    self.console.print(renderable, overflow="ignore", crop=False, style=style)
  File "/Users/joshua.grefte/Library/Python/3.10/lib/python/site-packages/pip/_vendor/rich/console.py", line 1752, in print
    extend(render(renderable, render_options))
  File "/Users/joshua.grefte/Library/Python/3.10/lib/python/site-packages/pip/_vendor/rich/console.py", line 1390, in render
    for render_output in iter_render:
  File "/Users/joshua.grefte/Library/Python/3.10/lib/python/site-packages/pip/_internal/utils/logging.py", line 134, in __rich_console__
    for line in lines:
  File "/Users/joshua.grefte/Library/Python/3.10/lib/python/site-packages/pip/_vendor/rich/segment.py", line 245, in split_lines
    for segment in segments:
  File "/

In [2]:
#https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/chatgpt?pivots=programming-language-chat-completions
#https://www.youtube.com/watch?v=uCKH8bmPgFs&t=502s

In [3]:
#create resource group and resource and deploy gpt4 model then use api key and endpoint value

In [9]:
import openai
openai.api_type = "azure"
openai.api_version = "2023-05-15" 
openai.api_base = "https://gpt4resource.openai.azure.com/" # Your Azure OpenAI resource's endpoint value.
openai.api_key = "618bf90290544e54b47bcfc2dba743da"

response = openai.ChatCompletion.create(
    #gpt-35-turbo max tokens = 4,096
    engine="test-gpt-35-turbo", # The deployment name you chose when you deployed the ChatGPT or GPT-4 model.
    messages=[
        {"role": "system", "content": "Assistant is an intelligent chatbot designed to help users create programming training materials"},
        {"role": "user", "content": "Could you please provide me with incorrect code in Node.js, followed by a corresponding unit test that will fail due to the introduced bug? At the end provide the corrected code. Area of interest is web development."}
    ],
    temperature = 0 # This is the degree of randomness of the model's output
)

#print(response)

output = response['choices'][0]['message']['content']
print('output: ', output)
print(50*'-')
print('tokens: ', response['usage'])

output:  Sure, here's an example:

Incorrect code:

```javascript
// server.js

const express = require('express');
const app = express();

app.get('/', (req, res) => {
  res.send('Hello World!');
});

app.listen(3000, () => {
  console.log('Server started on port 3000');
});
```

The bug in this code is that the route handler for the root path (`/`) is not returning a response with the correct content type. Instead of setting the `Content-Type` header to `text/html`, it is using the default content type of `text/plain`.

To write a failing unit test for this bug, we can use the `supertest` library to make a request to the server and check the response headers:

```javascript
// server.test.js

const request = require('supertest');
const app = require('./server');

describe('GET /', () => {
  it('should return HTML content', async () => {
    const response = await request(app).get('/');
    expect(response.status).toBe(200);
    expect(response.headers['content-type']).toMatch(/text\/

In [19]:
#SAVE OUTPUT IN A TXT FILE
import os

folder_path = 'txt_outputs'
text_files = [f for f in os.listdir(folder_path) if f.endswith('.txt')]
txt_number_list = []
for file_name in text_files:
    txt_number = file_name[-1]
    txt_number = int(txt_number)
    txt_number_list.append(txt_number)
if not txt_number_list: #check if list is empty, i.e. no txt files yet
    new_txt_number = str(1)
else:
    new_txt_number = str(max(txt_number_list) + 1)
text_file = open('txt_outputs/output' + '_' + new_txt_number + '.txt', "w")
text_file.write(output)
text_file.close()